<a href="https://colab.research.google.com/github/afeld/python-public-policy/blob/master/lecture_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NYU Wagner - Python Coding for Public Policy**
# Class 6

## Agenda

1. Guest lecture
1. Real-world natural language processing example
1. Government/coding Q&A

## [Vyki Englert](https://www.linkedin.com/in/vykienglert/)

- Former Software developer, now a "consultant", cyclist, and aspiring policymaker
- Co-founded [Compiler LA](https://compiler.la/), "a civically oriented strategy and software consultancy"
- Worked with the City of Santa Monica to:
  - Launch their open data initiative
  - Publish the city's first open dataset
  - Launch first website update in 14 years for Planning and Development Department
- Current Projects/ Affliations
  - [Policy Club](https://policyclub.io/)
  - [Data + Donuts LA](https://datadonuts.la/), a speaker series about people using technology and data in government
  - [Abundant Housing LA](https://abundanthousingla.org/) Policy Committee
  - Advisor at [CityGrows](https://citygro.ws/)

## Erica Hobby

Natural language processing (NLP) in the real world

## Me!

[![Screenshot of C2/Communicart](img/c2.png)](https://web.archive.org/web/20190124112324/https://cap.18f.gov/)

## [ATO Sprinting Team](https://18f.gsa.gov/2018/07/19/taking-the-ato-process-from-6-months-to-30-days/)

![Scatter chart with trendline showing decrease in ATO times](https://18f.gsa.gov/assets/blog/ato/ato-completion-date-graph.png)

## Agile budgeting

![Jellybeans](https://upload.wikimedia.org/wikipedia/commons/3/3e/Jellybeans.jpg)

## See also

[![Video still of man wearing suit at a microphone](https://img.youtube.com/vi/g-h6CtSwk30/maxresdefault.jpg)](https://www.youtube.com/watch?v=g-h6CtSwk30)

[Colleague explaining agile software development to the Michigan Senate](https://www.youtube.com/watch?v=g-h6CtSwk30)

## Questions!

- Working as a software developer
- Working in government
- Bread…?

## [Homework 6](https://colab.research.google.com/github/afeld/python-public-policy/blob/master/hw_5.ipynb#scrollTo=DRpVqnixIeFu)

I want your notebook to read like a narrative/diary/logbook.

## Thank you!

- alf9@nyu.edu
- aidan.feldman@gmail.com
- [@aidanfeldman](https://twitter.com/aidanfeldman/) on Twitter
- Come to [Hacker Hours](https://hackerhours.org/)!